In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import xarray as xr
import os

In [8]:
df = {}
file_path = os.path.join(os.getcwd(), "forex-data")
for f in os.listdir(file_path):
    
    df[f] = pd.read_csv(os.path.join(file_path, f))
    df[f]['openTime'] = pd.to_datetime(df[f].Date, unit='ns')
    df[f]['openTime'] = pd.DatetimeIndex(df[f]['openTime'] )
    df[f].index = df[f]['openTime']
    df[f].drop(columns=['openTime', 'Date'])
    df[f] = xr.DataArray(df[f], dims=['openTime', 'feature'])
    
        
chart = xr.Dataset(df).to_array(dim='asset')
print(chart)
print(type(chart)) # chart is the input matrix

<xarray.DataArray (asset: 7, openTime: 74551, feature: 9)>
array([[[0.81767, nan, ..., 14, Timestamp('2015-01-01 22:00:00')],
        [0.8173199999999999, nan, ..., 18, Timestamp('2015-01-01 22:10:00')],
        ...,
        [0.7207600000000001, nan, ..., 11, Timestamp('2016-12-30 21:40:00')],
        [0.72012, nan, ..., 79, Timestamp('2016-12-30 21:50:00')]],

       [[1.5583, nan, ..., 19, Timestamp('2015-01-01 22:00:00')],
        [1.55836, nan, ..., 7, Timestamp('2015-01-01 22:10:00')],
        ...,
        [1.23248, nan, ..., 493, Timestamp('2016-12-30 21:40:00')],
        [1.2341799999999998, nan, ..., 541, Timestamp('2016-12-30 21:50:00')]],

       ...,

       [[6.15319, nan, ..., 73, Timestamp('2015-01-01 22:00:00')],
        [6.15458, nan, ..., 287, Timestamp('2015-01-01 22:10:00')],
        ...,
        [7.0689, nan, ..., 448, Timestamp('2016-12-30 21:40:00')],
        [7.06722, nan, ..., 1409, Timestamp('2016-12-30 21:50:00')]],

       [[119.765, 79.765, ..., 23, Timestam

In [4]:
close_price = chart.sel(feature='Close')
close_price = close_price.T
usd_data = np.ones((close_price.shape[0],1))
in_data = np.hstack((usd_data, close_price))
in_data = in_data[1:] / in_data[:-1]
in_data = in_data / in_data[0]

# print(in_data)
m,n = in_data.shape

import logging

logging.basicConfig(level=logging.INFO)


In [7]:
import tensorflow.keras as keras

model = keras.Sequential([
    keras.layers.LSTM(12, return_sequences=False, return_state=False, unroll=True, input_shape=(n,1,1)),
    keras.layers.Dense(n,activation = 'relu'),
    keras.layers.Dropout((0.35)),
    keras.layers.Dense(n, activation='softmax')
    
])


import tensorflow.keras.backend as K

def loss6(y, b):
    gross_return = K.log(K.sum(y * b))
    return K.mean(gross_return)

model.compile(optimizer='adam',
              loss=loss6,
              metrics=['accuracy'])

# model.fit(in_data[:,:,None,None])
#The output is weight(b) of each period. vaule = b.*X
#X is the price matrix
model.predict(in_data[:,:,None,None])

array([[[[0.1958395 , 0.2711394 , 0.24062814, 0.14686893, 0.14552402]],

        [[0.17487976, 0.33281288, 0.2295879 , 0.13392219, 0.12879726]],

        [[0.17487976, 0.33281288, 0.2295879 , 0.13392219, 0.12879726]],

        [[0.17487976, 0.33281288, 0.2295879 , 0.13392219, 0.12879726]],

        [[0.15407258, 0.37736225, 0.21471731, 0.14042248, 0.11342537]]],


       [[[0.19584084, 0.27113634, 0.24062856, 0.1468684 , 0.14552589]],

        [[0.17487419, 0.33282536, 0.22958541, 0.13392127, 0.12879373]],

        [[0.17487693, 0.33280963, 0.2295788 , 0.13393177, 0.12880296]],

        [[0.17488979, 0.3327693 , 0.22958039, 0.1339427 , 0.1288178 ]],

        [[0.15409175, 0.37729368, 0.21472034, 0.14044403, 0.11345023]]],


       [[[0.19584122, 0.27113545, 0.24062866, 0.14686824, 0.14552642]],

        [[0.17487104, 0.33283153, 0.22958335, 0.13392167, 0.12879245]],

        [[0.17488426, 0.33279085, 0.22958016, 0.13393536, 0.12880935]],

        [[0.17488834, 0.3327768 , 0.22958486, 0